## labels analysis

In [ ]:
# load orginal data(COVID-19 TREC-IS 2020 task) from json file.
data_folder = 'data' + os.sep + 'TREC_IS_2020' + os.sep + 'COVID_19'
# TREC-IS A run data set
cv19_dc = pd.read_json( data_folder + os.sep + 'cv19_dc_test.json',lines=True,orient='records')
cv19_ws = pd.read_json(data_folder + os.sep + 'cv19_washington_state_test.json',lines=True,orient='records')
cv19_ny = pd.read_json(data_folder + os.sep + 'cv19_nyc_test.json',lines=True,orient='records')
cv19_a_run_info_labeled = pd.read_csv(data_folder + os.sep + 'cv19_TREC_2020_all_labeled_A.csv',
                           converters={"priority": lambda x: x.strip("[]").replace("'", "").split(", "),
                                       "categories": lambda x: x.strip("[]").replace("'", "").split(", ")})
cv19_dc_labeled = pd.merge(cv19_dc,cv19_a_run_info_labeled,on=['id'])
cv19_ws_labeled = pd.merge(cv19_ws,cv19_a_run_info_labeled,on=['id'])
cv19_ny_labeled = pd.merge(cv19_ny,cv19_a_run_info_labeled,on=['id'])
temp = [cv19_dc_labeled,cv19_ny_labeled,cv19_ws_labeled ]
cv19_a_run_info_labeled =  pd.concat(temp)

NameError: ignored

In [ ]:

with open(data_folder + os.sep + 'b_label2a.json') as f:
    labels_b_2a = json.load(f)

with open(data_folder + os.sep + 'b_label3.json') as f:
    labels_b_3 = json.load(f)

with open(data_folder + os.sep + 'b_label4.json') as f:
    labels_b_4 = json.load(f)

with open(data_folder + os.sep + 'b_label5.json') as f:
    labels_b_5 = json.load(f)

with open(data_folder + os.sep + 'b_label2b.json') as f:
    labels_b_2b = json.load(f)


labels21 = pd.DataFrame(labels_b_2a['events'][10]['tweets'][:])
labels22 = pd.DataFrame(labels_b_2a['events'][11]['tweets'][:])

labels31 = pd.DataFrame(labels_b_3['events'][7]['tweets'][:])
labels32 = pd.DataFrame(labels_b_3['events'][8]['tweets'][:])
labels33 = pd.DataFrame(labels_b_3['events'][12]['tweets'][:])
labels34 = pd.DataFrame(labels_b_3['events'][13]['tweets'][:])
# labels32 = pd.DataFrame(labels3['events'][2]['tweets'][:])

labels41 = pd.DataFrame(labels_b_4['events'][2]['tweets'][:])
labels42 = pd.DataFrame(labels_b_4['events'][3]['tweets'][:])
labels43 = pd.DataFrame(labels_b_4['events'][4]['tweets'][:])

labels51 = pd.DataFrame(labels_b_5['events'][8]['tweets'][:])
labels52 = pd.DataFrame(labels_b_5['events'][9]['tweets'][:])
labels53 = pd.DataFrame(labels_b_5['events'][10]['tweets'][:])
labels54 = pd.DataFrame(labels_b_5['events'][11]['tweets'][:])
labels55 = pd.DataFrame(labels_b_5['events'][12]['tweets'][:])
labels56 = pd.DataFrame(labels_b_5['events'][13]['tweets'][:])
labels57 = pd.DataFrame(labels_b_5['events'][17]['tweets'][:])
labels58 = pd.DataFrame(labels_b_5['events'][18]['tweets'][:])



# Union them all into one dataFrame
temp = [labels21, labels22, labels31, labels32, labels33, labels34, labels41, labels42, labels43, labels51,
          labels52, labels53, labels54, labels55, labels56, labels57, labels58]
all_labels_b = pd.concat(temp)
all_labels_b = all_labels_b.rename(columns={'postID': 'id'})
all_labels_b = all_labels_b.astype({"id": np.int64})
all_labels_b.reset_index()

In [ ]:
all_labels_union_b = []
# For debugging
count = 0
# Iterate all tweets including duplicate labels
for id in all_labels_b.id.values:
    priority = []
    categories = []
    count += 1
    print(count)
    # concatenate all labels for information type and prioritization task
    for idx in range(len(all_labels_b)):
      if id == all_labels_b.iloc[idx].id:
        priority += [all_labels_b.iloc[idx].priority]
        categories += all_labels_b.iloc[idx].categories

#     print(list(set(priority)))
    all_labels_union_b.append({'id': id,'full_text':all_labels_b.iloc[idx].text, 'priority': list(set(priority)), 'categories': list(set(categories))})
# Transer list to dataFrame
all_labels_union_b = pd.DataFrame(all_labels_union_b)
all_labels_union_b.to_csv('cache/all_labeled_duplicated_b.csv')
all_labels_union_b.drop_duplicates('id', keep='first', inplace=True)
all_labels_union_b.to_csv('cache/all_labeled_b.csv')

In [ ]:
frame = all_labels_union_b.copy()

In [ ]:
frame.index = np.arange(len(frame))
categories = ['Low', 'Medium', 'High', 'Critical']
for idx in range(len(frame)):
    length = len(frame.loc[idx, 'priority'])
    final_label = None
    print(idx)

    if length >= 3:
        number_count = [0, 0, 0, 0]
        for priority_label in frame.loc[idx, 'priority']:
            if priority_label == 'Low':
                number_count[0] += 1
            elif priority_label == 'Medium':
                number_count[1] += 1
            elif priority_label == 'High':
                number_count[2] += 1
            else:
                number_count[3] += 1
        final_label = categories[np.argmax(np.array(number_count))]
    else:
        final_label = frame.loc[idx, 'priority'][0]
        # print(final_label)
        frame.loc[idx, 'priority'] = final_label
        
cv19_b_run_labeled = frame

In [ ]:
cv19_b_run_labeled

In [ ]:
tmp = all_labels_b.groupby('id').count().reset_index()
ids = tmp[tmp['text'] >= 2]['id']
print(len(ids))
consist_num  = 0
for id in ids:
  labels = []
  for label in all_labels_b[all_labels_b['id'] == id].priority:
    labels.append(label)
  consist = len(set(labels))
  if consist == 1:
    consist_num += 1
consist_num 

459


203

In [ ]:
labels = []
for label in all_labels_b[all_labels_b['id'] == 1296130571672997889].priority:
  labels.append(label)
len(set(labels)) == 1

True

## Load data

In [ ]:
import os
from covid_tools import *
from BERTs import *

In [ ]:
# load orginal data(COVID-19 TREC-IS 2020 task) from json file.
data_folder = 'data' + os.sep + 'TREC_IS_2020' + os.sep + 'COVID_19'
# TREC-IS A run data set
cv19_dc = pd.read_json( data_folder + os.sep + 'cv19_dc_test.json',lines=True,orient='records')
cv19_ws = pd.read_json(data_folder + os.sep + 'cv19_washington_state_test.json',lines=True,orient='records')
cv19_ny = pd.read_json(data_folder + os.sep + 'cv19_nyc_test.json',lines=True,orient='records')
cv19_a_run_info_labeled = pd.read_csv(data_folder + os.sep + 'cv19_TREC_2020_all_labeled_A.csv',
                           converters={"priority": lambda x: x.strip("[]").replace("'", "").split(", "),
                                       "categories": lambda x: x.strip("[]").replace("'", "").split(", ")})
cv19_dc_labeled = pd.merge(cv19_dc,cv19_a_run_info_labeled,on=['id'])
cv19_ws_labeled = pd.merge(cv19_ws,cv19_a_run_info_labeled,on=['id'])
cv19_ny_labeled = pd.merge(cv19_ny,cv19_a_run_info_labeled,on=['id'])

cv19_dc_labeled = Process_labels.extract_majority_vote_label(cv19_dc_labeled)
cv19_ny_labeled = Process_labels.extract_majority_vote_label(cv19_ny_labeled)
cv19_ws_labeled = Process_labels.extract_majority_vote_label(cv19_ws_labeled)

temp = [cv19_dc_labeled,cv19_ny_labeled,cv19_ws_labeled ]
cv19_a_run_info_labeled =  pd.concat(temp)

cv19_a_run_info_labeled = cv19_a_run_info_labeled.reset_index()

cv19_a_run_info_labeled = Process_labels.extract_hashtags(cv19_a_run_info_labeled)

In [ ]:
cv19_b_run_info_labeled = pd.read_csv(data_folder + os.sep + 'cv19_TREC_2020_all_labeled_B.csv',
                           converters={"priority": lambda x: x.strip("[]").replace("'", "").split(", "),
                                       "categories": lambda x: x.strip("[]").replace("'", "").split(", ")})

cv19_Houston_2020 = pd.read_json(data_folder + os.sep + 'cv_Houston2020.json',lines=True,orient='records')
cv19_Atlanta_2020 = pd.read_json(data_folder + os.sep + 'cv_Atlanta2020.json',lines=True,orient='records')
cv19_Melbourne2020 = pd.read_json(data_folder + os.sep + 'cv_Melbourne2020.json',lines=True,orient='records')
cv19_NYC_2020 = pd.read_json(data_folder + os.sep + 'cv_NYC2020.json',lines=True,orient='records')
cv19_acksonville_2020 = pd.read_json(data_folder + os.sep + 'cv_Jacksonville2020.json',lines=True,orient='records')

# Merge original dataset with lablled dataset based on id of tweets
cv19_Houston_2020_labeled = pd.merge(cv19_Houston_2020, cv19_b_run_info_labeled,on=['id'])
cv19_Atlanta_2020_labeled = pd.merge(cv19_Atlanta_2020, cv19_b_run_info_labeled,on=['id'])
cv19_Melbourne2020_labeled = pd.merge(cv19_Melbourne2020, cv19_b_run_info_labeled,on=['id'])
cv19_NYC_2020_labeled = pd.merge(cv19_NYC_2020, cv19_b_run_info_labeled, on=['id'])
cv19_acksonville_2020_labeled = pd.merge(cv19_acksonville_2020,cv19_b_run_info_labeled, on=['id'])

temp = [cv19_Houston_2020_labeled,cv19_Atlanta_2020_labeled,cv19_Melbourne2020_labeled,cv19_NYC_2020_labeled, cv19_acksonville_2020_labeled ]
cv19_b_run_info_labeled =  pd.concat(temp)
cv19_b_run_info_labeled = cv19_b_run_info_labeled.reset_index()

cv19_b_run_info_labeled = Process_labels.extract_majority_vote_label(cv19_b_run_info_labeled)


cv19_b_run_info_labeled = Process_labels.extract_hashtags(cv19_b_run_info_labeled)

In [ ]:
cv19_b_run_info_labeled = cv19_b_run_info_labeled.rename(columns={'full_text_x':'full_text'})
categories = ['GoodsServices','InformationWanted','Volunteer','EmergingThreats','NewSubEvent','ServiceAvailable','Advice',]

# # Extract training set
# featureList = ['full_text'] #,'favorited'
# dataset_inputs = extract_features(Tweets,featureList)

# Extract labels
labelList = ['categories']
a_info_num_labels = Process_labels.extract_features(cv19_a_run_info_labeled,labelList)['categories']
b_info_num_labels = Process_labels.extract_features(cv19_b_run_info_labeled,labelList)['categories']
# Transfer a list of text labels into a list of number labels, like [0,1,0,0,0,0,0]
a_info_num_labels = Process_labels.extractLabels(cv19_a_run_info_labeled,a_info_num_labels)
b_info_num_labels = Process_labels.extractLabels(cv19_b_run_info_labeled,b_info_num_labels )

In [ ]:
a_priority_num_labels = cv19_a_run_info_labeled.priority
b_priority_num_labels = cv19_b_run_info_labeled.priority

In [ ]:
categories = ['GoodsServices','InformationWanted','Volunteer','EmergingThreats','NewSubEvent','ServiceAvailable','Advice']
# Sperate labels for each category and store them in one dict
# A dict Contains labels for all categories e.g. trainL_single[category]
labels_single_a = Process_labels.build_single_label(a_info_num_labels,categories)
labels_single_b = Process_labels.build_single_label(b_info_num_labels, categories)

Currrent is processing on categorie GoodsServices

Currrent is processing on categorie InformationWanted

Currrent is processing on categorie Volunteer

Currrent is processing on categorie EmergingThreats

Currrent is processing on categorie NewSubEvent

Currrent is processing on categorie ServiceAvailable

Currrent is processing on categorie Advice

Currrent is processing on categorie GoodsServices

Currrent is processing on categorie InformationWanted

Currrent is processing on categorie Volunteer

Currrent is processing on categorie EmergingThreats

Currrent is processing on categorie NewSubEvent

Currrent is processing on categorie ServiceAvailable

Currrent is processing on categorie Advice



# info classification

In [ ]:
#preprossesing
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# from nltk.tokenize import word_tokenize

# Use Spacy as nlp tool
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words
# Do not remove 'not' as stopword
if 'not' in all_stopwords:
  all_stopwords.remove('not')
if 'no' in all_stopwords:
  all_stopwords.remove('no')


# Speed up the processing of Spacy
disable=['ner']
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')
# These three functions are learnt from course named Text as data
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#@Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if (not token.is_stop):
        if ((token.is_alpha or token.is_digit or token.like_url)):
            normalized_tokens.append(normalized)
  return normalized_tokens

#@Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

count_vect = CountVectorizer(tokenizer=tokenize_normalize)
Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_normalize,max_features=25000,ngram_range=(1,2),sublinear_tf=(1,2))
    

## Cross event validtation

#### TF-idf and count vectorizer

In [ ]:
count_vect = CountVectorizer(tokenizer=tokenize_normalize)
Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_normalize,max_features=25000,ngram_range=(1,2),sublinear_tf=(1,2))

In [ ]:
a_batch = [cv19_a_run_info_labeled, labels_single_a]
b_batch = [cv19_b_run_info_labeled, labels_single_b]
best_over_sample_factors = [24,44,7,4,4,4,1]

for i in range(6, 7):
    section_category = i
    increase_time = best_over_sample_factors[i]
    Classical_classifiers.cross_event_validate_lr_info_task(section_category, a_batch, b_batch,increase_time, full_text_vectorizer=Tfidf_vect, hash_tags_vectorizer=count_vect
                        )


## SVM

#### 1. tf-idf + count

In [ ]:
train_features = cv19_a_run_info_labeled[:int(len(cv19_a_run_info_labeled)*0.8)]

test_features = cv19_a_run_info_labeled[int(len(cv19_a_run_info_labeled)*0.8) :]

featureList = ['full_text','hashtags','favorite_count'] #,'favorited'
train_features = Process_labels.extract_features(train_features,featureList)


# best_over_sample_factors = [24,44,7,4,4,4,1]
best_over_sample_factors = [24,44,7,4,4,4,1]
vectorizers = [Tfidf_vect , count_vect]
for i in range(7):
    print("Currently is processing %d" %(i))
    section_category = i
    train_labels = labels_single_a[section_category][:int(len(cv19_a_run_info_labeled)*0.8)]
    test_labels = labels_single_a[section_category][int(len(cv19_a_run_info_labeled)*0.8) :]
    running_set = [train_features, train_labels, test_features, test_labels]
    Classical_classifiers.svm(running_set, vectorizers, section_category, over_sample_targets=[1], increase_factor=best_over_sample_factors[i],
                        )

Currently is processing 0
Over-sample data set
feature-union
Train with feature union


c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appd

Cross validation: Average training set result: test_accuracy=0.997362 test_precision=0.000000 test_f1=0.000000 test_recall=0.000000 .

Cross validation: Average validation set result: train_accuracy=0.997362 train_precision=0.000000 train_f1=0.000000 train_recall=0.000000 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.999 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      1.000     0.999     0.999      1517
           1      0.000     0.000     0.000         0

    accuracy                          0.999      1517
   macro avg      0.500     0.499     0.500      1517
weighted avg      1.000     0.999     0.999      1517

Simple test result:
Prediction result: {'accuracy': 0.998681608437706, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     1.0000    0.9987    0.9993      1517
           1     0.0000    0.0000    

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\

Cross validation: Average training set result: test_accuracy=0.937519 test_precision=0.200000 test_f1=0.004938 test_recall=0.002500 .

Cross validation: Average validation set result: train_accuracy=0.941202 train_precision=0.689327 train_f1=0.113550 train_recall=0.061875 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.998 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      0.999     0.999     0.999      1516
           1      0.000     0.000     0.000         1

    accuracy                          0.998      1517
   macro avg      0.500     0.499     0.500      1517
weighted avg      0.999     0.998     0.998      1517

Simple test result:
Prediction result: {'accuracy': 0.998022412656559, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     0.9993    0.9987    0.9990      1516
           1     0.0000    0.0000    

...

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appd

Cross validation: Average training set result: test_accuracy=0.934718 test_precision=0.000000 test_f1=0.000000 test_recall=0.000000 .

Cross validation: Average validation set result: train_accuracy=0.934883 train_precision=0.800000 train_f1=0.005031 train_recall=0.002524 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.871 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      1.000     0.871     0.931      1517
           1      0.000     0.000     0.000         0

    accuracy                          0.871      1517
   macro avg      0.500     0.435     0.465      1517
weighted avg      1.000     0.871     0.931      1517

Simple test result:
Prediction result: {'accuracy': 0.8707976268951879, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     1.0000    0.8708    0.9309      1517
           1     0.0000    0.0000   

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\py

Cross validation: Average training set result: test_accuracy=0.877747 test_precision=0.400000 test_f1=0.005016 test_recall=0.002524 .

Cross validation: Average validation set result: train_accuracy=0.877747 train_precision=1.000000 train_f1=0.005036 train_recall=0.002525 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.870 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      0.999     0.871     0.931      1516
           1      0.000     0.000     0.000         1

    accuracy                          0.870      1517
   macro avg      0.500     0.435     0.465      1517
weighted avg      0.999     0.870     0.930      1517

Simple test result:
Prediction result: {'accuracy': 0.8701384311140409, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     0.9992    0.8707    0.9306      1516
           1     0.0000    0.0000   

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


#### 2. count + count

In [ ]:
vectorizers = [count_vect , count_vect]
for i in range(7):
    section_category = i
    train_labels = labels_single_a[section_category][:int(len(cv19_a_run_info_labeled)*0.8)]
    test_labels = labels_single_a[section_category][int(len(cv19_a_run_info_labeled)*0.8) :]
    running_set = [train_features, train_labels, test_features, test_labels]
    Classical_classifiers.svm(running_set, vectorizers, section_category, over_sample_targets=[1], increase_factor=best_over_sample_factors[i],
                        )

Over-sample data set
feature-union
Train with feature union


c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appd

Cross validation: Average training set result: test_accuracy=0.997362 test_precision=0.000000 test_f1=0.000000 test_recall=0.000000 .

Cross validation: Average validation set result: train_accuracy=0.997362 train_precision=0.000000 train_f1=0.000000 train_recall=0.000000 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.999 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      1.000     0.999     0.999      1517
           1      0.000     0.000     0.000         0

    accuracy                          0.999      1517
   macro avg      0.500     0.499     0.500      1517
weighted avg      1.000     0.999     0.999      1517

Simple test result:
Prediction result: {'accuracy': 0.998681608437706, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     1.0000    0.9987    0.9993      1517
           1     0.0000    0.0000    

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\py

Cross validation: Average training set result: test_accuracy=0.937519 test_precision=0.200000 test_f1=0.004938 test_recall=0.002500 .

Cross validation: Average validation set result: train_accuracy=0.941202 train_precision=0.689327 train_f1=0.113550 train_recall=0.061875 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.998 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      0.999     0.999     0.999      1516
           1      0.000     0.000     0.000         1

    accuracy                          0.998      1517
   macro avg      0.500     0.499     0.500      1517
weighted avg      0.999     0.998     0.998      1517

Simple test result:
Prediction result: {'accuracy': 0.998022412656559, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     0.9993    0.9987    0.9990      1516
           1     0.0000    0.0000    

...

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appd

Cross validation: Average training set result: test_accuracy=0.934718 test_precision=0.000000 test_f1=0.000000 test_recall=0.000000 .

Cross validation: Average validation set result: train_accuracy=0.934883 train_precision=0.800000 train_f1=0.005031 train_recall=0.002524 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.871 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      1.000     0.871     0.931      1517
           1      0.000     0.000     0.000         0

    accuracy                          0.871      1517
   macro avg      0.500     0.435     0.465      1517
weighted avg      1.000     0.871     0.931      1517

Simple test result:
Prediction result: {'accuracy': 0.8707976268951879, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     1.0000    0.8708    0.9309      1517
           1     0.0000    0.0000   

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\py

Cross validation: Average training set result: test_accuracy=0.877747 test_precision=0.400000 test_f1=0.005016 test_recall=0.002524 .

Cross validation: Average validation set result: train_accuracy=0.877747 train_precision=1.000000 train_f1=0.005036 train_recall=0.002525 .

Evaluating on test set:
Evaluation for: SVMresult on test set
Classifier 'SVMresult on test set' has Acc=0.870 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      0.999     0.871     0.931      1516
           1      0.000     0.000     0.000         1

    accuracy                          0.870      1517
   macro avg      0.500     0.435     0.465      1517
weighted avg      0.999     0.870     0.930      1517

Simple test result:
Prediction result: {'accuracy': 0.8701384311140409, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     0.9992    0.8707    0.9306      1516
           1     0.0000    0.0000   

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


#### Using wrod2vec embedding

In [ ]:
import gensim.downloader
glove_model = gensim.downloader.load('glove-twitter-200')

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("glove.twitter.27B.200d.txt")

In [ ]:
vectors = [glove_model[sen] for sen in sentences]

In [ ]:
a_batch = [cv19_a_run_info_labeled, labels_single_a]
b_batch = [cv19_b_run_info_labeled, labels_single_b]
best_over_sample_factors = [24,44,7,4,4,4,1]

for i in range(7):
    section_category = i
    increase_time = best_over_sample_factors[i]
    train_labels = labels_single_a[section_category][:int(len(cv19_a_run_info_labeled)*0.8)]
    test_labels = labels_single_a[section_category][int(len(cv19_a_run_info_labeled)*0.8) :]
    running_set = [train_features, train_labels, test_features, test_labels]
    Classical_classifiers.cross_event_validate_lr_info_task(section_category, a_batch, b_batch,increase_time, full_text_vectorizer=glove_model, hash_tags_vectorizer=count_vect
                        )

### Decision tree


#### 1. tf-idf + count

In [ ]:
train_features = cv19_a_run_info_labeled[:int(len(cv19_a_run_info_labeled)*0.8)]

test_features = cv19_a_run_info_labeled[int(len(cv19_a_run_info_labeled)*0.8) :]

featureList = ['full_text','hashtags','favorite_count'] #,'favorited'
train_features = Process_labels.extract_features(train_features,featureList)


# best_over_sample_factors = [24,44,7,4,4,4,1]
best_over_sample_factors = [24,44,7,4,4,4,1]
vectorizers = [Tfidf_vect , count_vect]
total_training_time = 0
for i in range(7):
    print("Currently is processing %d" %(i))
    section_category = i
    train_labels = labels_single_a[section_category][:int(len(cv19_a_run_info_labeled)*0.8)]
    test_labels = labels_single_a[section_category][int(len(cv19_a_run_info_labeled)*0.8) :]
    running_set = [train_features, train_labels, test_features, test_labels]
    total_training_time += Classical_classifiers.DecisionTreeCf(running_set, vectorizers, section_category, over_sample_targets=[1], increase_factor=best_over_sample_factors[i],
                        )

Currently is processing 0
Train with feature union


c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\si

Cross validation: Average training set result: test_accuracy=0.996373 test_precision=0.000000 test_f1=0.000000 test_recall=0.000000 .

Cross validation: Average validation set result: train_accuracy=1.000000 train_precision=1.000000 train_f1=1.000000 train_recall=1.000000 .

Evaluating on test set:
Evaluation for: DTresult on test set
Classifier 'DTresult on test set' has Acc=0.999 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      1.000     0.999     0.999      1517
           1      0.000     0.000     0.000         0

    accuracy                          0.999      1517
   macro avg      0.500     0.499     0.500      1517
weighted avg      1.000     0.999     0.999      1517

Simple test result:
Prediction result: {'accuracy': 0.998681608437706, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     1.0000    0.9987    0.9993      1517
           1     0.0000    0.0000    0.

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Train with feature union


...

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Train with feature union
Cross validation: Average training set result: test_accuracy=0.823096 test_precision=0.219859 test_f1=0.168791 test_recall=0.230032 .

Cross validation: Average validation set result: train_accuracy=1.000000 train_precision=1.000000 train_f1=1.000000 train_recall=1.000000 .

Evaluating on test set:
Evaluation for: DTresult on test set
Classifier 'DTresult on test set' has Acc=0.862 P=0.112 R=0.386 F1=0.174
              precision    recall  f1-score   support

           0      0.974     0.881     0.925      1460
           1      0.112     0.386     0.174        57

    accuracy                          0.862      1517
   macro avg      0.543     0.633     0.549      1517
weighted avg      0.941     0.862     0.897      1517

Simple test result:
Prediction result: {'accuracy': 0.8622280817402769, 'precision': 0.11224489795918367, 'f1': 0.17391304347826086, 'recall': 0.38596491228070173}
              precision    recall  f1-score   support

           0     0.

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Cross validation: Average training set result: test_accuracy=0.867344 test_precision=0.676525 test_f1=0.772459 test_recall=1.000000 .

Cross validation: Average validation set result: train_accuracy=1.000000 train_precision=1.000000 train_f1=1.000000 train_recall=1.000000 .

Evaluating on test set:
Evaluation for: DTresult on test set
Classifier 'DTresult on test set' has Acc=0.860 P=0.097 R=0.352 F1=0.152
              precision    recall  f1-score   support

           0      0.974     0.879     0.924      1463
           1      0.097     0.352     0.152        54

    accuracy                          0.860      1517
   macro avg      0.535     0.615     0.538      1517
weighted avg      0.942     0.860     0.896      1517

Simple test result:
Prediction result: {'accuracy': 0.8602504943968359, 'precision': 0.09693877551020408, 'f1': 0.152, 'recall': 0.35185185185185186}
              precision    recall  f1-score   support

           0     0.9735    0.8790    0.9239      1463
    

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


#### 2. count + count

In [ ]:
train_features = cv19_a_run_info_labeled[:int(len(cv19_a_run_info_labeled)*0.8)]

test_features = cv19_a_run_info_labeled[int(len(cv19_a_run_info_labeled)*0.8) :]

featureList = ['full_text','hashtags','favorite_count'] #,'favorited'
train_features = Process_labels.extract_features(train_features,featureList)


# best_over_sample_factors = [24,44,7,4,4,4,1]
best_over_sample_factors = [24,44,7,4,4,4,1]
vectorizers = [count_vect , count_vect]
total_training_time = 0
for i in range(7):
    print("Currently is processing %d" %(i))
    section_category = i
    train_labels = labels_single_a[section_category][:int(len(cv19_a_run_info_labeled)*0.8)]
    test_labels = labels_single_a[section_category][int(len(cv19_a_run_info_labeled)*0.8) :]
    running_set = [train_features, train_labels, test_features, test_labels]
    total_training_time += Classical_classifiers.DecisionTreeCf(running_set, vectorizers, section_category, over_sample_targets=[1], increase_factor=best_over_sample_factors[i],
                        )

Currently is processing 0
Train with feature union


c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Cross validation: Average training set result: test_accuracy=0.995879 test_precision=0.000000 test_f1=0.000000 test_recall=0.000000 .

Cross validation: Average validation set result: train_accuracy=1.000000 train_precision=1.000000 train_f1=1.000000 train_recall=1.000000 .

Evaluating on test set:
Evaluation for: DTresult on test set
Classifier 'DTresult on test set' has Acc=0.997 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      0.999     0.999     0.999      1515
           1      0.000     0.000     0.000         2

    accuracy                          0.997      1517
   macro avg      0.499     0.499     0.499      1517
weighted avg      0.997     0.997     0.997      1517

Simple test result:
Prediction result: {'accuracy': 0.997363216875412, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     0.9987    0.9987    0.9987      1515
           1     0.0000    0.0000    0.

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Cross validation: Average training set result: test_accuracy=0.989767 test_precision=0.958748 test_f1=0.890065 test_recall=0.875000 .

Cross validation: Average validation set result: train_accuracy=1.000000 train_precision=1.000000 train_f1=1.000000 train_recall=1.000000 .

Evaluating on test set:
Evaluation for: DTresult on test set
Classifier 'DTresult on test set' has Acc=0.997 P=0.000 R=0.000 F1=0.000
              precision    recall  f1-score   support

           0      0.998     0.999     0.998      1514
           1      0.000     0.000     0.000         3

    accuracy                          0.997      1517
   macro avg      0.499     0.499     0.499      1517
weighted avg      0.996     0.997     0.996      1517

Simple test result:
Prediction result: {'accuracy': 0.996704021094265, 'precision': 0.0, 'f1': 0.0, 'recall': 0.0}
              precision    recall  f1-score   support

           0     0.9980    0.9987    0.9983      1514
           1     0.0000    0.0000    0.

...

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Train with feature union
Cross validation: Average training set result: test_accuracy=0.816006 test_precision=0.211559 test_f1=0.148348 test_recall=0.214937 .

Cross validation: Average validation set result: train_accuracy=1.000000 train_precision=1.000000 train_f1=1.000000 train_recall=1.000000 .

Evaluating on test set:
Evaluation for: DTresult on test set
Classifier 'DTresult on test set' has Acc=0.854 P=0.117 R=0.319 F1=0.172
              precision    recall  f1-score   support

           0      0.963     0.880     0.920      1445
           1      0.117     0.319     0.172        72

    accuracy                          0.854      1517
   macro avg      0.540     0.600     0.546      1517
weighted avg      0.923     0.854     0.884      1517

Simple test result:
Prediction result: {'accuracy': 0.8536585365853658, 'precision': 0.11734693877551021, 'f1': 0.17164179104477612, 'recall': 0.3194444444444444}
              precision    recall  f1-score   support

           0     0.9

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Cross validation: Average training set result: test_accuracy=0.853877 test_precision=0.634652 test_f1=0.742449 test_recall=1.000000 .

Cross validation: Average validation set result: train_accuracy=1.000000 train_precision=1.000000 train_f1=1.000000 train_recall=1.000000 .

Evaluating on test set:
Evaluation for: DTresult on test set
Classifier 'DTresult on test set' has Acc=0.842 P=0.148 R=0.287 F1=0.195
              precision    recall  f1-score   support

           0      0.945     0.882     0.913      1416
           1      0.148     0.287     0.195       101

    accuracy                          0.842      1517
   macro avg      0.547     0.585     0.554      1517
weighted avg      0.892     0.842     0.865      1517

Simple test result:
Prediction result: {'accuracy': 0.8424522083058669, 'precision': 0.14795918367346939, 'f1': 0.19528619528619529, 'recall': 0.2871287128712871}
              precision    recall  f1-score   support

           0     0.9455    0.8821    0.9127  

c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
c:\users\longkuku\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass beta=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
